In [ ]:
from models import dc_D, dc_G, dc_d, dc_g, GoodDiscriminator, GoodGenerator, GoodDiscriminatord
from VisionData import GAN_trainer
import torch
from torchvision import transforms
from torchvision.datasets import CIFAR10, MNIST
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
torch.backends.cudnn.benchmark = True

# MNIST

In [ ]:
def transform(x):
    x = transforms.ToTensor()(x)
    return (x - 0.5) / 0.5
data_root = '/datas/mnist'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
learning_rate = 0.0001
z_dim = 96
D = dc_D()
G = dc_G(z_dim=z_dim)
dataset = MNIST(data_root, train=True, transform=transform)
trainer = GAN_trainer(D=D, G=G, device=device, dataset=dataset, z_dim=z_dim, batchsize=128, lr=learning_rate, show_iter=200,
                         weight_decay=0.0, d_penalty=0.0, g_penalty=0, noise_shape=(64, z_dim), gp_weight=0)
trainer.train_gd(epoch_num=50, mode='Adam', logname='MNIST2', dataname='MNIST')

In [ ]:
trainer.load_checkpoint('checkpoints/0.00000MNIST-0.0001/0.00010_50000.pth', count=50000, load_d=True, load_g=True)
# load_d: True load discriminator; load_g: True load generator

In [ ]:
trainer.traing(epoch_num=5, mode='Adam', logname='MNIST3', dataname='MNIST')
# train generator with discriminator fixed

In [ ]:
trainer.train_d(epoch_num=100, mode='Adam', logname='MNIST2', dataname='MNIST')
# train discriminator with generator fixed

In [ ]:
trainer.train_ocgd(epoch_num=50, update_D=True, collect_info=True, logname='MNIST3', dataname='MNIST')
# train one side CGD:
# update_D: True: train discriminator with generator fixed
# update_D: False: train generator with discriminator fixed

# CIFAR 10

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
learning_rate = 0.0001
batch_size = 64
z_dim = 128
dropout = None
# Dropout: None or 0.5
if dropout is not None:
    print('dropout!')
    D = GoodDiscriminatord(dropout=dropout)
else:
    D = GoodDiscriminator()
G = GoodGenerator()
dataset = CIFAR10(root='datas/cifar10', download=True, train=True, transform=transform)
trainer = GAN_trainer(D=D, G=G, device=device, dataset=dataset, z_dim=z_dim, batchsize=batch_size, 
                      lr=learning_rate, show_iter=500, weight_decay=0.0, d_penalty=0.0, g_penalty=0, 
                      noise_shape=(64, z_dim), gp_weight=0)
trainer.train_bcgd(epoch_num=300, mode='MCGD', collect_info=True, dataname='CIFAR10-WGAN', 
                   logname='CIFAR10-WGAN', loss_type='WGAN')
# loss_type : WGAN, JSD